In [1]:
import pandas as pd

unemp_county = pd.read_csv("D:\d Documents\datasets for practice\output.csv")

unemp_county.head()

,Year,Month,State,County,Rate
0,2015,February,Mississippi,Newton County,6.1
1,2015,February,Mississippi,Panola County,9.4
2,2015,February,Mississippi,Monroe County,7.9
3,2015,February,Mississippi,Hinds County,6.1
4,2015,February,Mississippi,Kemper County,10.6


### economists argue that setting a min wage means less people get employed. This is what we are trying to see
### We don't have exact data as we have state minimum wages, but the min wages may differ across counties in each state. Still, we are going to try and map each county to a state and compare the rates!

In [2]:
# from part 4

df = pd.read_csv("D:\d Documents\datasets for practice\minwage.csv")

act_min_wage = pd.DataFrame()

# we can iterate over all the groups too
for name, group in df.groupby("State"):
    if act_min_wage.empty:
        act_min_wage = group.set_index("Year")[["Low.2018"]].rename(columns = {"Low.2018" : name})
    
    else:
        act_min_wage = act_min_wage.join(group.set_index("Year")[["Low.2018"]].rename(columns = {"Low.2018" : name}))

act_min_wage.head() # alabama will have all 0s because of NaNs

# remember how we dropped NaNs for min_wage_corr? lets do the same for the act_min_wage
import numpy as np
act_min_wage = act_min_wage.replace(0, np.NaN).dropna(axis = 1)
act_min_wage.tail()

,Alaska,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Federal (FLSA),Guam,Hawaii,...,Pennsylvania,Puerto Rico,Rhode Island,South Dakota,Utah,Vermont,Washington,West Virginia,Wisconsin,Wyoming
Year,,,,,,,,,,,,,,,,,,,,,
2013,8.33,6.72,8.60,8.36,8.87,7.79,8.87,7.79,7.79,7.79,...,7.79,5.46,8.33,7.79,7.79,9.24,9.88,7.79,7.79,5.54
2014,8.20,6.61,9.52,8.46,9.20,8.20,10.05,7.67,7.67,7.67,...,7.67,5.37,8.46,7.67,7.67,9.23,9.86,7.67,7.67,5.45
2015,9.24,7.92,9.51,8.69,9.67,8.72,11.09,7.66,8.72,8.19,...,7.66,5.37,9.51,8.98,7.66,9.67,10.00,8.45,7.66,5.44
2016,10.17,8.35,10.43,8.67,10.02,8.61,12.00,7.56,8.61,8.87,...,7.56,5.30,10.02,8.92,7.56,10.02,9.88,9.13,7.56,5.37
2017,10.01,8.68,10.22,9.50,10.32,8.43,11.75,7.41,8.43,9.45,...,7.41,5.19,9.81,8.84,7.41,10.22,11.24,8.94,7.41,5.26


### now how would we see the r/s bw unemployment and min wage? We should try to add a column called min wage on our unemp_county and then maybe see if there is a corr?

### let's first create a functions that returns us the min wage based on state and year!

In [3]:
%%time

def get_min_wage(year, state):
    try:
        return act_min_wage.loc[year][state]
    except:
        return np.NaN 

get_min_wage(2012, "Colorado") # finds the min wage of Colorado in they year 2012 using the act_min_wage df!

Wall time: 999 µs


8.33

### now map this function to unemp_county! 
### this will take quite awhile, so let's time it and do other stuff while waiting

In [8]:
%%time 
# unemp_county['min_wage'] = (map(get_min_wage, unemp_county['Year'], unemp_county['State']))
unemp_county['min_wage'] = list(map(get_min_wage, unemp_county['Year'], unemp_county['State']))



Wall time: 2min 16s


## Why do you keep getting NaN for min_wage???
### get_min_wage() works, map() returns a map object in min_wage but list(map()) gets you NaNs???

### answer: because that's how the function was formed... Look through the entire list and you can see that NOT ALL are NaNs!

In [12]:
# unemp_county['min_wage'] = 1
unemp_county['min_wage']
#unemp_county.head()


0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
5          NaN
6          NaN
7          NaN
8          NaN
9          NaN
10         NaN
11         NaN
12         NaN
13         NaN
14         NaN
15         NaN
16         NaN
17         NaN
18         NaN
19         NaN
20         NaN
21         NaN
22         NaN
23         NaN
24         NaN
25         NaN
26         NaN
27         NaN
28         NaN
29         NaN
          ... 
885518    7.64
885519    7.64
885520    7.64
885521    7.64
885522    7.64
885523    7.64
885524    7.64
885525    7.64
885526    7.64
885527    7.64
885528    7.64
885529    7.64
885530    7.64
885531    7.64
885532    8.46
885533    8.46
885534    8.46
885535    8.46
885536    8.46
885537    8.46
885538    8.46
885539    8.46
885540    8.46
885541    8.46
885542    8.46
885543    8.46
885544    8.46
885545    8.46
885546    8.46
885547    8.46
Name: min_wage, Length: 885548, dtype: float64

In [15]:
# troubleshooting

len(unemp_county['State']) # 885548 ie 885547 is the last row!

(unemp_county['Year'][885547], unemp_county['State'][885547]) # this works
get_min_wage(2009, 'Maine') # we can get min wage what we found

act_min_wage.loc[2009]['Maine'] # we can do it without the function
act_min_wage.loc[(unemp_county['Year'][885547])][(unemp_county['State'][885547])]

8.46

In [21]:
unemp_county[["Rate", "min_wage"]].corr()


,Rate,min_wage
Rate,1.000000,0.153047
min_wage,0.153047,1.000000


In [22]:
unemp_county.dropna()[["Rate", "min_wage"]].cov()


,Rate,min_wage
Rate,9.670952,0.651586
min_wage,0.651586,1.874228


## Rookie mistake 1: "Rate, min_wage" vs "rate", "min_wage"....

## Rookie mistake 2: don't just check the head, look at the entire data or also look at the tail!. They might not all be NaN!!!

In [23]:
pres16 = pd.read_csv("D:\d Documents\datasets for practice\pres16results.csv")
pres16.head()

,county,fips,cand,st,pct_report,votes,total_votes,pct,lead
0,NaN,US,Donald Trump,US,0.9951,60350241.0,127592176.0,0.472993,Donald Trump
1,NaN,US,Hillary Clinton,US,0.9951,60981118.0,127592176.0,0.477938,Donald Trump
2,NaN,US,Gary Johnson,US,0.9951,4164589.0,127592176.0,0.032640,Donald Trump
3,NaN,US,Jill Stein,US,0.9951,1255968.0,127592176.0,0.009844,Donald Trump
4,NaN,US,Evan McMullin,US,0.9951,451636.0,127592176.0,0.003540,Donald Trump


In [24]:
pres16.tail()

,county,fips,cand,st,pct_report,votes,total_votes,pct,lead
18470,Yauco Municipio,72153,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18471,Midway Islands,74300,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18472,St. Croix Island,78010,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18473,St. John Island,78020,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18474,St. Thomas Island,78030,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### doesn't seem like all counties are inputted into this dataset

### in any case, let's find the top 10 candidates

In [30]:
top10yourversion = pres16['cand'][1:10].values

print(top10yourversion)

['Hillary Clinton' 'Gary Johnson' 'Jill Stein' 'Evan McMullin'
 'Darrell Castle' 'Gloria La Riva' 'Rocky De La Fuente'
 ' None of these candidates' 'Richard Duncan']


In [31]:
top_candidates = pres16.head(10)['cand'].values
print(top_candidates)

['Donald Trump' 'Hillary Clinton' 'Gary Johnson' 'Jill Stein'
 'Evan McMullin' 'Darrell Castle' 'Gloria La Riva' 'Rocky De La Fuente'
 ' None of these candidates' 'Richard Duncan']


In [59]:
county_2015 = unemp_county.copy()[(unemp_county["Year"] == 2015) & (unemp_county["Month"] == "February")] # added copy() to prevent future errors!
county_2015.head()

,Year,Month,State,County,Rate,min_wage
0,2015,February,Mississippi,Newton County,6.1,NaN
1,2015,February,Mississippi,Panola County,9.4,NaN
2,2015,February,Mississippi,Monroe County,7.9,NaN
3,2015,February,Mississippi,Hinds County,6.1,NaN
4,2015,February,Mississippi,Kemper County,10.6,NaN


### he realises there are NaNs now lol

In [60]:
pres16['st'].unique()

array(['US', 'CA', 'FL', 'TX', 'NY', 'PA', 'IL', 'OH', 'MI', 'NC', 'GA',
       'VA', 'NJ', 'MA', 'WI', 'MN', 'MO', 'WA', 'IN', 'CO', 'TN', 'MD',
       'SC', 'AL', 'AZ', 'LA', 'KY', 'OR', 'CT', 'IA', 'OK', 'MS', 'KS',
       'NV', 'AR', 'UT', 'NE', 'NM', 'ME', 'NH', 'WV', 'ID', 'MT', 'RI',
       'DE', 'HI', 'SD', 'ND', 'VT', 'DC', 'WY', 'AK', nan], dtype=object)

### as you can see, pres16 states are postal codes while those of unemp_county are full names. so let's change that!

In [41]:
state_abbv = pd.read_csv("D:\d Documents\datasets for practice\state_abbv.csv", index_col=0) # ie letting the name become the index

In [45]:
state_abbv.head()
state_abbv = state_abbv[["Postal Code"]] # this means that we only want the postal code AND THE INDEX WHICH IS THE NAME OF THE STATES!

In [46]:
state_abbv.head() 

,Postal Code
State/District,
Alabama,AL
Alaska,AK
Arizona,AZ
Arkansas,AR
California,CA


In [49]:
state_abbv_dict = state_abbv.to_dict()['Postal Code'] # remember the issure from last time about the dict being in Postal code!
state_abbv_dict

{'Alabama': 'AL',
 'Alaska': 'AK',
 'Arizona': 'AZ',
 'Arkansas': 'AR',
 'California': 'CA',
 'Colorado': 'CO',
 'Connecticut': 'CT',
 'Delaware': 'DE',
 'District of Columbia': 'DC',
 'Florida': 'FL',
 'Georgia': 'GA',
 'Hawaii': 'HI',
 'Idaho': 'ID',
 'Illinois': 'IL',
 'Indiana': 'IN',
 'Iowa': 'IA',
 'Kansas': 'KS',
 'Kentucky': 'KY',
 'Louisiana': 'LA',
 'Maine': 'ME',
 'Maryland': 'MD',
 'Massachusetts': 'MA',
 'Michigan': 'MI',
 'Minnesota': 'MN',
 'Mississippi': 'MS',
 'Missouri': 'MO',
 'Montana': 'MT',
 'Nebraska': 'NE',
 'Nevada': 'NV',
 'New Hampshire': 'NH',
 'New Jersey': 'NJ',
 'New Mexico': 'NM',
 'New York': 'NY',
 'North Carolina': 'NC',
 'North Dakota': 'ND',
 'Ohio': 'OH',
 'Oklahoma': 'OK',
 'Oregon': 'OR',
 'Pennsylvania': 'PA',
 'Rhode Island': 'RI',
 'South Carolina': 'SC',
 'South Dakota': 'SD',
 'Tennessee': 'TN',
 'Texas': 'TX',
 'Utah': 'UT',
 'Vermont': 'VT',
 'Virginia': 'VA',
 'Washington': 'WA',
 'West Virginia': 'WV',
 'Wisconsin': 'WI',
 'Wyoming': 'WY

In [61]:
county_2015['State'] = county_2015['State'].map(state_abbv_dict)

In [62]:
county_2015.head()

,Year,Month,State,County,Rate,min_wage
0,2015,February,MS,Newton County,6.1,NaN
1,2015,February,MS,Panola County,9.4,NaN
2,2015,February,MS,Monroe County,7.9,NaN
3,2015,February,MS,Hinds County,6.1,NaN
4,2015,February,MS,Kemper County,10.6,NaN


### he seems to get a warning for not copying but you don't? In any case, you get no warning but IF YOU RUN THE CODE TWICE, ALL YOUR STATES BECOMES NaNs!!!

### see line 59 for the updated code!

In [64]:
print(len(county_2015))
print(len(pres16))

2802
18475


## merge() and join() have the same implementation but different defaults!!!

### we want to merge county_2015 with pres16 based on State and County. We need to make the column names identical. Let's fix that first

In [66]:
county_2015.head() # has "County" and "State"

,Year,Month,State,County,Rate,min_wage
0,2015,February,MS,Newton County,6.1,NaN
1,2015,February,MS,Panola County,9.4,NaN
2,2015,February,MS,Monroe County,7.9,NaN
3,2015,February,MS,Hinds County,6.1,NaN
4,2015,February,MS,Kemper County,10.6,NaN


In [69]:
pres16.tail() # has "county" and "st". observe how they are all lower case vs county_2015

,county,fips,cand,st,pct_report,votes,total_votes,pct,lead
18470,Yauco Municipio,72153,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18471,Midway Islands,74300,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18472,St. Croix Island,78010,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18473,St. John Island,78020,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18474,St. Thomas Island,78030,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
pres16.rename(columns = {'county':"County", "st":"State"}, inplace = True)
pres16.head()

,County,fips,cand,State,pct_report,votes,total_votes,pct,lead
0,NaN,US,Donald Trump,US,0.9951,60350241.0,127592176.0,0.472993,Donald Trump
1,NaN,US,Hillary Clinton,US,0.9951,60981118.0,127592176.0,0.477938,Donald Trump
2,NaN,US,Gary Johnson,US,0.9951,4164589.0,127592176.0,0.032640,Donald Trump
3,NaN,US,Jill Stein,US,0.9951,1255968.0,127592176.0,0.009844,Donald Trump
4,NaN,US,Evan McMullin,US,0.9951,451636.0,127592176.0,0.003540,Donald Trump


## joining by index and multiple indexes
### the easiest way to join is by index. We can actually have more than 1 index ie county and state!

In [72]:
for df in [county_2015, pres16]:
    df.set_index(["County", "State"], inplace = True)

In [76]:
pres16.head(10)

fips                       cand  pct_report       votes  \
County State                                                           
NaN    US      US               Donald Trump      0.9951  60350241.0   
       US      US            Hillary Clinton      0.9951  60981118.0   
       US      US               Gary Johnson      0.9951   4164589.0   
       US      US                 Jill Stein      0.9951   1255968.0   
       US      US              Evan McMullin      0.9951    451636.0   
       US      US             Darrell Castle      0.9951    180877.0   
       US      US             Gloria La Riva      0.9951     48308.0   
       US      US         Rocky De La Fuente      0.9951     32120.0   
       US      US   None of these candidates      0.9951     28824.0   
       US      US             Richard Duncan      0.9951     23501.0   

              total_votes       pct          lead  
County State                                       
NaN    US     127592176.0  0.472993  Donald Trump  
       US     127592176.0  0.477938  Donald Trump  
       US     127592176.0  0.032640  Donald Trump  
       US     127592176.0  0.009844  Donald Trump  
       US     127592176.0  0.003540  Donald Trump  
       US     127592176.0  0.001418  Donald Trump  
       US     127592176.0  0.000379  Donald Trump  
       US     127592176.0  0.000252  Donald Trump  
       US     127592176.0  0.000226  Donald Trump  
       US     127592176.0  0.000184  Donald Trump

### we are going to focus on just Trump for now. their r/s to min wage might be different for each candidate eg libertarians ==> no min wage. But let's see the main diff bw trump and hilary

In [85]:
# only run this once!
# pres16 = pres16[pres16['cand'] == "Donald Trump"] # filters out only for Trump
pres16 = pres16[["pct"]] # only want the "pct" column as a DATAFRAME
pres16.dropna(inplace = True)
pres16.head()

pct
County State          
NaN    US     0.472993
       CA     0.330641
       FL     0.490640
       TX     0.525830
       NY     0.374752

In [88]:
# merging them all together

all_together = county_2015.merge(pres16, on=["County", "State"])
all_together.dropna(inplace=True)
all_together.head()

,,Year,Month,Rate,min_wage,pct
County,State,,,,,
Major County,OK,2015,February,2.6,2.11,0.864960
Pottawatomie County,OK,2015,February,4.5,2.11,0.701342
Johnston County,OK,2015,February,6.5,2.11,0.770057
Jefferson County,OK,2015,February,5.0,2.11,0.812367
Beaver County,OK,2015,February,2.8,2.11,0.888243


In [90]:
all_together.drop("Year", axis=1, inplace=True) # removes the COL ie axis = 1


KeyError: "['Year'] not found in axis"

In [91]:
all_together.corr()

,Rate,min_wage,pct
Rate,1.000000,0.186689,-0.085985
min_wage,0.186689,1.000000,-0.325036
pct,-0.085985,-0.325036,1.000000


In [92]:
all_together.cov()

,Rate,min_wage,pct
Rate,5.743199,0.683870,-0.031771
min_wage,0.683870,2.336451,-0.076602
pct,-0.031771,-0.076602,0.023772


### from what you can see, there seems to be a negative correl bw min wage and pct vote for trump. Which would make sense; min wage are more liberal.

### unemployment rate also does not seem to correl to pct vote

### in any case, most of these findings are not surprising nor significant. This is just a walkthrough of pandas data science!